In [131]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
pd.options.display.max_columns = 1000
import warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [132]:
item_categories = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/item_categories.csv", sep=",")
items = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/items.csv", sep=",")
shops = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/shops.csv", sep=",")
test = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/test.csv", sep=",")
sales_train = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\competitive-data-science-predict-future-sales/sales_train.csv", sep=",")

In [133]:
sales_train['date'] = pd.to_datetime(sales_train['date'])
sales_train['date_block_num'] = sales_train['date_block_num'] + 1
sales_train['month'] = sales_train['date_block_num']% 12
a = []
for i in sales_train['month']:
    if i == 0:
        a.append(i +12)
    else:
        a.append(i)
sales_train['month'] = a
sales_train['day'] = sales_train['date'].dt.day
sales_train['year'] = sales_train['date'].dt.year
sales_train['mm'] = sales_train['date'].dt.month
sales_train['day'] = ((sales_train['month'] - sales_train['day']) - sales_train['mm'])*-1
sales_train = sales_train.drop(['mm'],axis = 1)
df = pd.DataFrame({'year': sales_train['year'],
                   'month': sales_train['month'],
                   'day': sales_train['day']})

sales_train['date'] =  pd.to_datetime(df)
sales_train.to_csv('sales_train.csv', index=False)
print('File Successfully Saved.!!!')

File Successfully Saved.!!!


In [134]:
datebn = sales_train['date_block_num']
sales_train = sales_train.drop(['date_block_num','day'],axis = 1)
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 35
sales_train['date_block_num'] = datebn

In [135]:
sales_train

,date,shop_id,item_id,item_price,item_cnt_day,month,year,date_block_num
0,2013-01-02,59,22154,999.00,1.0,1,2013,1
1,2013-01-03,25,2552,899.00,1.0,1,2013,1
2,2013-01-05,25,2552,899.00,-1.0,1,2013,1
3,2013-01-06,25,2554,1709.05,1.0,1,2013,1
4,2013-01-15,25,2555,1099.00,1.0,1,2013,1
...,...,...,...,...,...,...,...,...
2935844,2015-10-10,25,7409,299.00,1.0,10,2015,34
2935845,2015-10-09,25,7460,299.00,1.0,10,2015,34
2935846,2015-10-14,25,7459,349.00,1.0,10,2015,34
2935847,2015-10-22,25,7440,299.00,1.0,10,2015,34


In [136]:
test

,ID,shop_id,item_id,month,year,date_block_num
0,0,5,5037,11,2015,35
1,1,5,5320,11,2015,35
2,2,5,5233,11,2015,35
3,3,5,5232,11,2015,35
4,4,5,5268,11,2015,35
...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,35
214196,214196,45,16188,11,2015,35
214197,214197,45,15757,11,2015,35
214198,214198,45,19648,11,2015,35


In [137]:
items_map = pd.Series(data=items['item_category_id'], index=items['item_id'])
sales_train['item_id_category'] = sales_train['item_id'].map(items_map)
test['item_id_category'] = test['item_id'].map(items_map)

In [138]:
itemid = list(sales_train.item_id)
price = list(sales_train.item_price)
shopid = list(sales_train.shop_id)

itemidt = list(test.item_id)
shopidt = list(test.shop_id)

ve = []
for i in range(len(sales_train)):
    ve.append(str(shopid[i])+str(itemid[i]))

sales_train['ve'] = ve
voo = list(sales_train['ve'].unique())
    
vet = []
for i in range(len(test)):
    vet.append(str(shopidt[i])+str(itemidt[i]))
test['ve'] = vet

sales_train = sales_train.drop(['item_price','date'],axis = 1)

In [139]:
sales_train

,shop_id,item_id,item_cnt_day,month,year,date_block_num,item_id_category,ve
0,59,22154,1.0,1,2013,1,37,5922154
1,25,2552,1.0,1,2013,1,58,252552
2,25,2552,-1.0,1,2013,1,58,252552
3,25,2554,1.0,1,2013,1,58,252554
4,25,2555,1.0,1,2013,1,56,252555
...,...,...,...,...,...,...,...,...
2935844,25,7409,1.0,10,2015,34,55,257409
2935845,25,7460,1.0,10,2015,34,55,257460
2935846,25,7459,1.0,10,2015,34,55,257459
2935847,25,7440,1.0,10,2015,34,57,257440


In [140]:
test

,ID,shop_id,item_id,month,year,date_block_num,item_id_category,ve
0,0,5,5037,11,2015,35,19,55037
1,1,5,5320,11,2015,35,55,55320
2,2,5,5233,11,2015,35,19,55233
3,3,5,5232,11,2015,35,23,55232
4,4,5,5268,11,2015,35,20,55268
...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,35,55,4518454
214196,214196,45,16188,11,2015,35,64,4516188
214197,214197,45,15757,11,2015,35,55,4515757
214198,214198,45,19648,11,2015,35,40,4519648


In [141]:
p = [int(i) for i in (list(test['ve']))]
v = [int(i) for i in (list(sales_train['ve']))]
sales_train['date_block_num'] = sales_train['date_block_num']/12
test['date_block_num'] = test['date_block_num']/12
sales_train['ve'] = v
test['ve'] = p
sales_train['ve'] = sales_train['ve']/1200
test['ve'] = test['ve']/1200

In [142]:
sales_train

,shop_id,item_id,item_cnt_day,month,year,date_block_num,item_id_category,ve
0,59,22154,1.0,1,2013,0.083333,37,4935.128333
1,25,2552,1.0,1,2013,0.083333,58,210.460000
2,25,2552,-1.0,1,2013,0.083333,58,210.460000
3,25,2554,1.0,1,2013,0.083333,58,210.461667
4,25,2555,1.0,1,2013,0.083333,56,210.462500
...,...,...,...,...,...,...,...,...
2935844,25,7409,1.0,10,2015,2.833333,55,214.507500
2935845,25,7460,1.0,10,2015,2.833333,55,214.550000
2935846,25,7459,1.0,10,2015,2.833333,55,214.549167
2935847,25,7440,1.0,10,2015,2.833333,57,214.533333


In [143]:
test

,ID,shop_id,item_id,month,year,date_block_num,item_id_category,ve
0,0,5,5037,11,2015,2.916667,19,45.864167
1,1,5,5320,11,2015,2.916667,55,46.100000
2,2,5,5233,11,2015,2.916667,19,46.027500
3,3,5,5232,11,2015,2.916667,23,46.026667
4,4,5,5268,11,2015,2.916667,20,46.056667
...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,2.916667,55,3765.378333
214196,214196,45,16188,11,2015,2.916667,64,3763.490000
214197,214197,45,15757,11,2015,2.916667,55,3763.130833
214198,214198,45,19648,11,2015,2.916667,40,3766.373333


In [144]:
submissionid = test['ID']
test = test.drop(['ID'],axis = 1)
traini = sales_train[:60000]
validation = sales_train[60000:65000]
Xval = validation.drop(['item_cnt_day'],axis = 1)
yval = validation.item_cnt_day
X = traini.drop(['item_cnt_day'],axis = 1)
y = traini.item_cnt_day

In [145]:
test

,shop_id,item_id,month,year,date_block_num,item_id_category,ve
0,5,5037,11,2015,2.916667,19,45.864167
1,5,5320,11,2015,2.916667,55,46.100000
2,5,5233,11,2015,2.916667,19,46.027500
3,5,5232,11,2015,2.916667,23,46.026667
4,5,5268,11,2015,2.916667,20,46.056667
...,...,...,...,...,...,...,...
214195,45,18454,11,2015,2.916667,55,3765.378333
214196,45,16188,11,2015,2.916667,64,3763.490000
214197,45,15757,11,2015,2.916667,55,3763.130833
214198,45,19648,11,2015,2.916667,40,3766.373333


In [146]:
X

,shop_id,item_id,month,year,date_block_num,item_id_category,ve
0,59,22154,1,2013,0.083333,37,4935.128333
1,25,2552,1,2013,0.083333,58,210.460000
2,25,2552,1,2013,0.083333,58,210.460000
3,25,2554,1,2013,0.083333,58,210.461667
4,25,2555,1,2013,0.083333,56,210.462500
...,...,...,...,...,...,...,...
59995,13,11231,1,2013,0.083333,40,1092.692500
59996,13,11235,1,2013,0.083333,40,1092.695833
59997,13,11249,1,2013,0.083333,40,1092.707500
59998,13,8067,1,2013,0.083333,55,115.055833


In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [148]:
def trainandsplit(x,y,xtest,ytest,xval,yval,test,submissionid):
        from sklearn.metrics import mean_squared_error
        from sklearn.linear_model import SGDClassifier
        from sklearn import linear_model
        reg = linear_model.LinearRegression()
        clf = SGDClassifier(max_iter=1000, tol=1e-3)
        clf.fit(x,y)
        reg.fit(x,y)
        predicted_score = clf.predict(xtest)
        predicted_scorev = clf.predict(xval)
        predicted_scorer = reg.predict(xtest)
        predicted_scorerv = reg.predict(xval)
        testr = clf.predict(test)
        testsdg = reg.predict(test)
        prediction = pd.DataFrame()
        predictionv = pd.DataFrame()
        prediction['sdgtrain'] = predicted_score
        prediction['regtrain'] = predicted_scorer
        predictionv['sdgvalid_train'] = predicted_scorev
        predictionv['regvalid_train'] = predicted_scorerv
        predictionvd = pd.DataFrame()
        predictionvd['sdgtest'] = testr
        predictionvd['regtest'] = testsdg
        clfg=xgb.XGBRegressor()
        reg.fit(prediction,ytest)
        reg.fit(predictionv,yval)
        p = reg.predict(prediction)
        d = reg.predict(predictionv)
        testf = reg.predict(predictionvd)
        error22 = mean_squared_error(ytest,p)
        error22v = mean_squared_error(yval,d)
        error = mean_squared_error(ytest,predicted_score)
        errorv = mean_squared_error(yval,predicted_scorev)
        error2 = mean_squared_error(ytest,predicted_scorer)
        error2v = mean_squared_error(yval,predicted_scorerv)
        sub = pd.DataFrame()
        sub['ID'] = submissionid
        sub['item_cnt_month'] = testf
        return clf,reg,error,errorv,error2,error2v,error22,error22v,sub

In [149]:
model1,model2,error,errorv,error2,error2v,error22,error22v,submission = trainandsplit(X_train,y_train,X_test,y_test,Xval,yval,test,submissionid)

In [150]:
error,errorv

(0.37313131313131315, 0.3066)

In [151]:
error2,error2v

(0.3533058835830731, 0.2912075666886667)

In [152]:
error22,error22v

(0.3568165507916051, 0.2888211707195745)

In [153]:
submission

,ID,item_cnt_month
0,0,1.179060
1,1,1.154581
2,2,1.179705
3,3,1.176878
4,4,1.179114
...,...,...
214195,214195,1.070536
214196,214196,1.056722
214197,214197,1.061655
214198,214198,1.085055


In [247]:
submission.to_csv('letsgooo555767.csv', index=False)
print('File Successfully Saved.!!!')

File Successfully Saved.!!!


In [207]:
submission['item_cnt_month']

0         1.040113
1         1.101767
2         1.038862
3         1.045920
4         1.040402
            ...   
214195    1.269454
214196    1.299774
214197    1.286659
214198    1.235396
214199    1.167139
Name: item_cnt_month, Length: 214200, dtype: float64